In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import time

In [2]:
def get_links(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    test = soup.find(id='div_sched_1631_1')
    x = "score"
    test = test.findAll("td", {"data-stat": x})
    link_list = []
    for i in test:
        i = i.find("a")
        if i is not None:
         link = i['href']
         link_list.append(link)
         #print(link)
    return link_list

In [3]:
#@title

#stats matchday
statsm = ["player", "shirtnumber", "nationality", "position", "age", "minutes", "goals", "assists", "touches", "sca",
          "gca", 'pens_made', 'shots_total', 'shots_on_target', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg',
          'xa', 'passes', 'passes_pct', 'progressive_passes', 'progressive_carries', 'dribbles_completed', 'dribbles']

#stats keeper matchday
statskeepers = ['player', 'shots_on_target_against', 'goals_against_gk', 'saves', 'save_pct', 'psxg_gk']


#Functions to get the data in a dataframe using BeautifulSoup

def get_tables_m(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    matchweek = soup.find(id='content').find("div").contents[1]

    team1_table = all_tables[0]
    team2_table = all_tables[7]
    gk1_table = all_tables[6]
    gk2_table = all_tables[13]
    gk1_table.append(gk2_table)
    team1_table.append(team2_table)

    return team1_table, gk1_table, matchweek


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    #print(rows_player)
    for row in rows_player:
        #print(row)
        if (row.find('th', {"scope": "row"}) != None):

            for f in features_wanted_player:
                #print(f)
                cell = row.find("td", {"data-stat": f})
                if cell is None:
                    text = row.find('th', {"scope": "row"}).find('a').contents[0]
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
                #print(cell)
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    #print(text)
                    if (text == ''):
                        text = '0'
                    if ((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') & (
                            f != 'birth_year')):
                        text = float(text.replace(',', ''))
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player




def frame_for_category(category, top, end, features):
    url = (top + category + end)
    print(url)
    player_table, gk_table, matchweek = get_tables_m(url)
    df_player = get_frame(features, player_table)
    df_gk = get_frame(statskeepers, gk_table)
    df_player['matchweek'] = matchweek
    df_gk['matchweek'] = matchweek
    return df_player, df_gk




def get_matchday_data(top, end, linklist):
    pl, gk = frame_for_category('/en/matches/e3c3ddf0', top, end, statsm)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl, tmpgk = frame_for_category(i, top, end, statsm)
        pl = pd.concat([pl, tmppl], axis=0)
        gk = pd.concat([gk, tmpgk], axis=0)
        time.sleep(5)


    return pl, gk

In [4]:
link_list = get_links('https://fbref.com/en/comps/9/1631/schedule/2017-2018-Premier-League-Scores-and-Fixtures')

# split list in 4 parts to not bring down website
list1 = []
list2 = []
list3 = []
list4 = []
count = 0

for i in link_list:
    if count < 100:
       list1.append(i)
    elif count< 200:
        list2.append(i)
    elif count< 300:
        list3.append(i)
    else:
        list4.append(i)
    count = count +1

In [8]:

df_1, gk_1 = get_matchday_data('https://fbref.com/', '', list1 )
df_2, gk_2 = get_matchday_data('https://fbref.com/', '', list2 )
df_3, gk_3 = get_matchday_data('https://fbref.com/', '', list3 )
df_4, gk_4 = get_matchday_data('https://fbref.com/', '', list4 )

#df_test, gk_test = get_matchday_data('https://fbref.com/', '', link_list )

https://fbref.com//en/matches/e3c3ddf0
https://fbref.com//en/matches/01f6b961/Stoke-City-Everton-March-17-2018-Premier-League
https://fbref.com//en/matches/134dc51c/Bournemouth-West-Bromwich-Albion-March-17-2018-Premier-League
https://fbref.com//en/matches/3b6ded14/Huddersfield-Town-Crystal-Palace-March-17-2018-Premier-League
https://fbref.com//en/matches/99d79931/Liverpool-Watford-March-17-2018-Premier-League
https://fbref.com//en/matches/98213b8d/Crystal-Palace-Liverpool-March-31-2018-Premier-League
https://fbref.com//en/matches/41e0dc54/Manchester-United-Swansea-City-March-31-2018-Premier-League
https://fbref.com//en/matches/683b188f/Brighton-and-Hove-Albion-Leicester-City-March-31-2018-Premier-League
https://fbref.com//en/matches/68df68a4/West-Bromwich-Albion-Burnley-March-31-2018-Premier-League
https://fbref.com//en/matches/781945f9/West-Ham-United-Southampton-March-31-2018-Premier-League
https://fbref.com//en/matches/f7c1128f/Newcastle-United-Huddersfield-Town-March-31-2018-Premi

In [15]:
def concatenate_files(f1, f2, f3, f4):
    total = pd.concat([f1, f2, f3, f4], axis=0)
    total = total.drop_duplicates()
    total.reset_index(inplace=True, drop=True)
    return total

gk_2017 = concatenate_files(gk_1, gk_2, gk_3, gk_4)
pl_2017 = concatenate_files(df_1, df_2, df_3, df_4)

#pl_2017
#df_test

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [11]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2017_keepers.csv',index=False)
    return df

create_csv(gk_2017)


,player,shots_on_target_against,goals_against_gk,saves,save_pct,psxg_gk,matchweek
0,Petr Čech,4.0,3.0,1.0,25.0,2.4,(Matchweek 1)
1,Kasper Schmeichel,10.0,4.0,6.0,60.0,3.0,(Matchweek 1)
2,Heurelho Gomes,4.0,3.0,2.0,50.0,3.1,(Matchweek 1)
3,Simon Mignolet,5.0,3.0,1.0,40.0,2.8,(Matchweek 1)
4,Wayne Hennessey,6.0,3.0,4.0,50.0,2.0,(Matchweek 1)
...,...,...,...,...,...,...,...
760,Heurelho Gomes,1.0,1.0,0.0,0.0,0.1,(Matchweek 38)
761,Alex McCarthy,2.0,1.0,1.0,50.0,0.6,(Matchweek 38)
762,Claudio Bravo,2.0,0.0,2.0,100.0,0.4,(Matchweek 38)
763,Hugo Lloris,9.0,4.0,5.0,55.6,3.7,(Matchweek 38)


In [129]:
#gk_2021[['player', 'matchweek']]

test_group = gk_2021.groupby(['player']).sum()['saves']
gk_strength = test_group.sort_values(ascending=False)
gk_strength

player
Illan Meslier          130.0
David de Gea           122.0
José Sá                119.0
Kasper Schmeichel      113.0
Jordan Pickford        107.0
Nick Pope               98.0
Robert Sánchez          96.0
Łukasz Fabiański        95.0
Tim Krul                93.0
Hugo Lloris             90.0
Emiliano Martínez       84.0
Vicente Guaita          79.0
Aaron Ramsdale          77.0
David Raya              68.0
Edouard Mendy           67.0
Fraser Forster          67.0
Martin Dúbravka         66.0
Alisson                 65.0
Ben Foster              56.0
Ederson                 54.0
Álvaro Fernández        46.0
Daniel Bachmann         39.0
Alex McCarthy           38.0
Angus Gunn              26.0
Karl Darlow             25.0
Kepa Arrizabalaga       11.0
Jack Butland            11.0
Bernd Leno               9.0
Freddie Woodman          9.0
Asmir Begović            7.0
Wayne Hennessey          7.0
Willy Caballero          6.0
Caoimhín Kelleher        5.0
Kristoffer Klaesson      4.0
Jason S

In [133]:
rslt_df = gk_2021[gk_2021['player'] == 'Bernd Leno']
rslt_df[['player', 'matchweek']]

,player,matchweek
1,Bernd Leno,(Matchweek 1)
36,Bernd Leno,(Matchweek 2)
41,Bernd Leno,(Matchweek 3)
573,Bernd Leno,(Matchweek 30)
